In [1]:
import numpy as np
import pandas as pd

import os
from dotenv import load_dotenv
from snowflake.snowpark import Session

In [6]:
load_dotenv()

True

In [7]:
ACCOUNT = os.getenv('ACCOUNT')
USER = os.getenv('SP_USER')
PASSWORD = os.getenv('PASSWORD')
ROLE = os.getenv('ROLE')
WAREHOUSE = os.getenv('WAREHOUSE')
DATABASE = os.getenv('DATABASE')
SCHEMA = os.getenv('SCHEMA')

print(ACCOUNT, USER, ROLE, WAREHOUSE, DATABASE, SCHEMA)

hum.us-east-2.aws ericatuva UVARND UVARND_WH CORE CLIENT


In [8]:
connection_parameters = {
    'account': os.getenv('ACCOUNT'),
    'user': USER,
    'password': os.getenv('PASSWORD'),
    'role': 'RUP',
    #'role': os.getenv('ROLE'),
    'warehouse': os.getenv('WAREHOUSE'),
    'database': os.getenv('DATABASE'),
    'schema': os.getenv('SCHEMA')
}

In [9]:
test_session = Session.builder.configs(connection_parameters).create()

In [10]:
events_query = """
SELECT 
    visitor_id
    , event
    , created
FROM event
WHERE DATE_TRUNC('year', day) = DATE('2022-01-01')
"""

In [11]:
print('querying...')
query_results = test_session.sql(events_query).collect()
print('query done')
test_session.close()
print('session closed')

querying...
query done
session closed


In [12]:
# convert to pandas df
query_json = list(map(lambda x: x.as_dict(), query_results))
query_df = pd.DataFrame(query_json)
query_df.to_csv('data.csv', index = 0)
# query_df = pd.read_csv('events_data.csv')

In [13]:
data = query_df
data

,VISITOR_ID,EVENT,CREATED
0,a8461296-6698-4287-8d32-27f6a40910bc,pageview,2022-06-17 01:50:40.525383
1,1df628fb-3f7f-4a86-8ea9-2cb13972f9cd,pageview,2022-10-23 09:16:27.306953
2,5b015762-8347-4ede-afff-325d7f30969e,post-read-end,2022-10-23 09:59:10.684398
3,dae8632d-03b5-4b61-8157-4b00cda6bbe4,post-read-end,2022-10-23 09:53:49.171193
4,20a79cfc-9f54-4e6a-a9e7-5ec25dce2447,pageview,2022-10-23 09:07:05.452466
...,...,...,...
10868387,c4f763b7-3f10-453e-ba7d-836594597cd5,post-read-start,2022-08-17 08:56:50.331150
10868388,ea430a65-3894-4a51-9aa5-054485e33ba9,pageview,2022-08-17 08:55:05.480175
10868389,82156c83-f68b-4d65-9c67-413361ec16d9,post-read-mid,2022-08-17 09:06:38.031812
10868390,d5b31ec1-67ba-4862-8f30-542cf2e28e24,post-read-end,2022-08-17 09:28:13.530097


In [14]:
data.dropna(inplace = True)
data

,VISITOR_ID,EVENT,CREATED
0,a8461296-6698-4287-8d32-27f6a40910bc,pageview,2022-06-17 01:50:40.525383
1,1df628fb-3f7f-4a86-8ea9-2cb13972f9cd,pageview,2022-10-23 09:16:27.306953
2,5b015762-8347-4ede-afff-325d7f30969e,post-read-end,2022-10-23 09:59:10.684398
3,dae8632d-03b5-4b61-8157-4b00cda6bbe4,post-read-end,2022-10-23 09:53:49.171193
4,20a79cfc-9f54-4e6a-a9e7-5ec25dce2447,pageview,2022-10-23 09:07:05.452466
...,...,...,...
10868387,c4f763b7-3f10-453e-ba7d-836594597cd5,post-read-start,2022-08-17 08:56:50.331150
10868388,ea430a65-3894-4a51-9aa5-054485e33ba9,pageview,2022-08-17 08:55:05.480175
10868389,82156c83-f68b-4d65-9c67-413361ec16d9,post-read-mid,2022-08-17 09:06:38.031812
10868390,d5b31ec1-67ba-4862-8f30-542cf2e28e24,post-read-end,2022-08-17 09:28:13.530097


In [15]:
len(np.unique(data[['VISITOR_ID']]))

1437685

In [16]:
users = data.VISITOR_ID.value_counts().loc[lambda x: x >= 100].reset_index()['index']
users

0       98719c0b-c648-4ea8-a0ca-1fd9ea458a4b
1       b75ae8f1-5e4b-4cbd-b2af-2e4e008f4951
2       a8c1ab55-e7f8-4ad9-8a3b-284e60944045
3       3a3a35f3-ad31-40b8-9bc8-03068da11532
4       a1c67daf-ad84-4cf2-be92-914dd1a5ceaa
                        ...                 
3625    738a430d-6cff-4076-ac87-c5f0cbcc296d
3626    b308ed5b-b49f-491c-868a-18163945cf4c
3627    30cac342-4d46-4030-ae6a-78906efdf24c
3628    ad6c2ef7-7565-456f-9ebc-26c5e2e5840e
3629    7e36a64a-1155-4aef-ab74-994511835953
Name: index, Length: 3630, dtype: object

In [ ]:
df = pd.DataFrame()
for user in users:
    temp = data[data.VISITOR_ID == user]
    arr = list(temp.sort_values('CREATED').iloc[:,1:].itertuples(index=False, name=None))
    dat = pd.DataFrame({'user': user, 'engagement': [arr]})
    df = pd.concat([df,dat])
    
df

In [ ]:
df.to_csv('RNNdata.csv', index = False)

In [ ]:
reread = pd.read_csv('RNNdata.csv')
reread